In [1]:
import numpy as np
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.spatial import distance
import re, sys, os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from IPython.display import display
import seaborn.apionly as sns
plt.style.use('ggplot')
sns.set_style('ticks')
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from torchvision import datasets
from IPython.display import Image
import h5py
import numpy

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Enable inline plotting  
%matplotlib inline

/Users/mahzadkhoshlessan/miniconda3/envs/ml/lib/python3.5/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/Users/mahzadkhoshlessan/miniconda3/envs/ml/lib/python3.5/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/Users/mahzadkhoshlessan/miniconda3/envs/ml/lib/python3.5/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
nn.Conv2d?

Init signature: nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
Docstring:     
Applies a 2D convolution over an input signal composed of several input
planes.

In the simplest case, the output value of the layer with input size
:math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
can be precisely described as:

.. math::
    \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
    \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


where :math:`\star` is the valid 2D `cross-correlation`_ operator,
:math:`N` is a batch size, :math:`C` denotes a number of channels,
:math:`H` is a height of input planes in pixels, and :math:`W` is
width in pixels.

* :attr:`stride` controls the stride for the cross-correlation, a single
  number or a tuple.

* :attr:`padding` controls the amount of implicit zero-paddings on both
  sides for

In [6]:
sys.path.append("/Users/mahzadkhoshlessan/Desktop/Machine-Learning/Intro-to-Deep-Learning/Final_Project/PyTorch")

In [7]:
from CNN_Model import project_01, normalize_im, NeuralNet, L1L2loss, weight_init, Checkpoint
from DataLoader import Dataset

In [8]:
# For reproducibility

In [9]:
np.random.seed(123)

In [10]:
# Load training & test data as training and validation sets

In [11]:
h5Train = h5py.File('../Input/TrainingSplit.h5', 'r')
h5Test = h5py.File('../Input/TestSplit.h5', 'r')

In [12]:
partition = {'train': list(h5Train.keys()), 'test': list(h5Test.keys())}

In [ ]:
P = dict()
P['numEpoch'] = 6
P['learning_rate'] = 0.001
P['batchSize'] = 16

In [ ]:
# Generators
training_set = Dataset(h5Train,P['batchSize'],partition['train'])
train_loader = DataLoader(training_set, shuffle=True)

validation_set = Dataset(h5Test,P['batchSize'],partition['test'])
test_loader = DataLoader(validation_set, shuffle=False)

In [ ]:
myModel = NeuralNet()

In [ ]:
psize = 208
optimizer = optim.Adam(myModel.parameters(), lr=P['learning_rate'])
criterion = L1L2loss((1, psize, psize))

In [ ]:
myModel.apply(weight_init)
scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=5, min_lr=0.00005)

In [ ]:
# Define the loss history recorder

In [ ]:
df = pd.DataFrame(columns=('epoch', 'loss', 'loss_test'))
Weight_file = os.path.abspath(os.path.normpath(os.path.join(os.getcwd(),'Checkpointing/Checkpointer.h5')))
meanstd_name = os.path.abspath(os.path.normpath(os.path.join(os.getcwd(),'meanstd_name.mat')))
path = os.path.abspath(os.path.normpath(os.path.join(os.getcwd(),'output.csv')))

In [ ]:
N_train = 7000
N_test = 3000

In [ ]:
loss_list = {'train':[],'test':[]}
loss_c = {'train':np.inf,'test':np.inf}
data_loader = {'train':train_loader,'test':test_loader}

In [30]:
loss_list = {'train':[],'test':[]} 

In [38]:
loss_list['test'].append(4.5) 

In [39]:
loss_list

{'test': [2.5, 3.5, 6.5, 5.5, 4.5], 'train': []}

In [40]:
is_best = bool(1.5 < min(loss_list['test']))  

In [41]:
is_best

True

In [ ]:
for epoch in range(P['numEpoch']):
    for phase in ['train', 'test']:
        if phase == 'train':
            myModel.train()
        else:
            myModel.eval()
    
        running_loss = 0.0
        counter = 0
        print(phase)
        for i, (images, labels) in enumerate(data_loader[phase]):
            # Run the forward pass
            counter = counter+1
            images = images.reshape(P['batchSize'], 1, psize, psize)
            labels = labels.reshape(P['batchSize'], 1, psize, psize) 
            
            density_pred = myModel(images)
            loss = criterion(labels,density_pred)
        
            if phase == 'train':
                scheduler.optimizer.zero_grad()
                loss.backward()
                scheduler.step(loss_c['test']) 

            running_loss += loss.detach().numpy()
            
            if phase=='train' and counter==400:
                break
            if phase=='test' and counter==187:
                break
                
        running_loss *= P['batchSize']/counter
        loss_list[phase].append(running_loss) 

        # Save the model weights after each epoch if the validation loss decreased
        loss_c[phase] = running_loss
        if phase=='test':
            if loss_list[phase]:
                is_best = bool(loss_c[phase] < min(loss_list[phase]))    
                Checkpoint(myModel, optimizer, epoch, is_best, Weight_file) 
    
        loss_list[phase].append(running_loss) 

    # B.3) stat for this epoch
    print(str(epoch)+'    loss:'+str(loss_c['train'])+' '+str(loss_c['test']))
    # save
    df.loc[epoch] = [epoch, loss_c['train'], loss_c['test']]
    df.to_csv(path)   

In [ ]:
mdict = {"mean_test": mean_val_test, "std_test": std_val_test}                  
sio.savemat(meanstd_name, mdict) 

In [ ]:
dff = pd.read_csv('keras-data/Keras_loss.csv', sep=r'\t',engine='python', header=None)

In [ ]:
dff1 = pd.read_csv('keras-data/Keras-loss-no-scheduler.csv', sep=r'\t',engine='python', header=None)

In [ ]:
dff2 = pd.read_csv('keras-data/output111.csv', sep=r',',engine='python')

In [ ]:
dff3 = pd.read_csv('keras-data/output.csv', sep=r',',engine='python')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(dff[0],dff[1],'-o')
ax.plot(dff[0],dff[2],'-o')

ax.plot(dff1[0],dff1[1],'-o')
ax.plot(dff1[0],dff1[2],'-o')

ax.plot(dff2['epoch'],dff2['loss'],'-o')
ax.plot(dff2['epoch'],dff2['loss_test'],'-o')

# ax.plot(dff3['epoch'],dff3['loss'],'-o')
# ax.plot(dff3['epoch'],dff3['loss_test'],'-o')

ax.grid()
ax.set_xlabel('epochs')
ax.set_ylabel('Loss')

ax.legend(['train loss Keras', 'test loss Keras','train loss Keras-No Scheduler', 'test loss Keras-No Scheduler',
           'train loss Torch', 'test loss Torch'])

fig.savefig('loss_out.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(dff3['epoch'],dff3['loss'],'-o')
ax.plot(dff3['epoch'],dff3['loss_test'],'-o')

ax.grid()
ax.set_xlabel('epochs')
ax.set_ylabel('Loss')

ax.legend(['train loss Torch-No Scheduler-No Init', 'test loss Torch-No Scheduler-No Init'])
fig.savefig('loss_out2.pdf')

In [ ]:
if cuda:
    checkpoint = torch.load(resume_weights)
else:
    # Load GPU model on CPU
    checkpoint = torch.load(resume_weights,
                            map_location=lambda storage,
                            loc: storage)
start_epoch = checkpoint['epoch']
best_accuracy = checkpoint['best_accuracy']
model.load_state_dict(checkpoint['state_dict'])
print("=> loaded checkpoint '{}' (trained for {} epochs)".format(resume_weights, checkpoint['epoch']))
